# Stock Sentiment Analysis from Headlines

More and more hedge funds and independent traders are utilizing data science to process the wealth of information available from news headlines in the quest for profit. In this project, I will generate investment insights by applying sentiment analysis on financial news headlines, webscraped from FINVIZ.com. Repsecting data science ethics with regard to webscraping, I have downloaded various HTML files for two large firms: Facebook & Tesla.

Through conducting sentiment analysis, we can examine the emotion behind the headlines and predict whether the market feels good or bad about a stock. Then, we can make educated guesses on how certain stocks will perform and trade accordingly. Below, we import these files:

In [1]:
# import libraries
from bs4 import BeautifulSoup
import os

html_tables = {}

# for every data set in os dataset folder
for table_name in os.listdir('datasets'):
    # filepath
    table_path = f'datasets/{table_name}'

    # open as read-only, read into 'html'
    table_file = open(table_path, 'r')
    html = BeautifulSoup(table_file)

    # add news-table to 'html_tables' dictionary
    html_table = html.find(id = 'news-table')
    html_tables[table_name] = html_table

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'datasets'